In [0]:
#need latest version to use generate()
!git clone https://github.com/huggingface/transformers.git
!cd transformers; pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 18706 (delta 13), reused 14 (delta 2), pack-reused 18670
Receiving objects: 100% (18706/18706), 11.10 MiB | 25.72 MiB/s, done.
Resolving deltas: 100% (13795/13795), done.
Processing /content/transformers
     |████████████████████████████████| 3.1MB 2.8MB/s 
     |████████████████████████████████| 1.0MB 28.1MB/s 
     |████████████████████████████████| 870kB 36.9MB/s 
  Created wheel for transformers: filename=transformers-2.3.0-cp36-none-any.whl size=460206 sha256=6936c2f0964ee25d688a7c4fe9640625450d5439ff28edeebb655438bfd880cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ymsywrgi/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=8d2a4c1cb9a25ea5a5c8325bb3a89867aacf390c4cfe49bb590ae3186175cb6e

In [0]:
import transformers
transformers.__version__

'2.3.0'

In [0]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

from typing import List
import pandas as pd
from google.colab import files, drive
from tqdm import tqdm_notebook

import torch
import numpy as np
from transformers import CTRLTokenizer, CTRLLMHeadModel, CTRLConfig

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [0]:
#mount gdrive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
tokenizer = CTRLTokenizer.from_pretrained('ctrl')
model = CTRLLMHeadModel.from_pretrained('ctrl').to(device)

In [0]:
def clean_text(text: str, review_star:str = '5.0') -> List[str]:
  input_context = f'Reviews Rating: {review_star}'
  input_context2 = input_context[8:]
  res = text.replace(f'{input_context} \n ','').split(f' \n {input_context2} \n ') #clean out multiple reviews
  res[-1] = '.'.join(res[-1].split('.')[:-1]) + '.' #clean out unfinished sentences
  #if last review only contains an unfinished sentence, skip it
  if res[-1] == '.':
    res = res[:-1]
  return res

def generate_text(review_star: str = '5.0', num_return_sequences = 60) -> str:
  input_context = f'Reviews Rating: {review_star}'
  input_context2 = input_context[8:]
  input_ids = torch.tensor(tokenizer.encode(input_context)).unsqueeze(0).to(device)  # encode input context
  outputs = model.generate(input_ids=input_ids, max_length=100, do_sample=True, num_beams=1, temperature=1., 
                           top_k=50, top_p=1., repetition_penalty=1.2, length_penalty=1., 
                           eos_token_ids = tokenizer.encode('|'), num_return_sequences=num_return_sequences) 
  texts = [tokenizer.decode(i, skip_special_tokens=True) for i in outputs.squeeze()]
  cleaned_texts = []
  for text in texts:
    cleaned_texts += clean_text(text,review_star)
  return cleaned_texts

def save_file(results:List[str], review_star:str ,i:int):
  fname = f'fake_reviews_{review_star}_{i}.csv'
  results_df = pd.DataFrame(results)
  results_df.columns = ['text']
  results_df.to_csv(fname, index=False)
  results_df.to_csv(f'/content/gdrive/My Drive/fake_reviews_data/{fname}', index=False)
  print(results_df.shape)
  # files.download(fname)

In [0]:
review_star = '5.0'
nb_rounds = 1000
results = []
for i in tqdm_notebook(range(nb_rounds)):
  results += generate_text(review_star, 60)
  if i%100==0:
    save_file(results,review_star,i)

(98, 1)
(9958, 1)
(19883, 1)
(29796, 1)
(39690, 1)
(49578, 1)


In [0]:
# #put in console
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)